In [464]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow.keras.layers import Input, Add, Dense, Activation, Dropout
from tensorflow.keras.models import Model, load_model, Sequential

In [465]:
df = pd.read_csv("train_transaction.csv")
df.shape

(590540, 394)

In [466]:
df.head(20)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [467]:
labels = df["isFraud"]

del df["isFraud"]

In [468]:
len(labels)

590540

### Data Wrangling for Transactions

In [469]:
#df.columns.tolist()

In [470]:
# TransactionDT
#df.TransactionDT.isna().sum()

df["DTSec"] = df.TransactionDT % 60
df["DTMin"] = (df.TransactionDT % 3600) // 60
df["DTHour"] = (df.TransactionDT % 86400) // 3600
df["DTDow"] = (df.TransactionDT % 604800) // 86400

del df['TransactionDT']

In [471]:
# TransactionAmt
#df.TransactionAmt.isna().sum()
# No changes at the beginning

In [472]:
# ProductCD
#df.ProductCD.isna().sum()
#df.ProductCD.value_counts()

df = pd.get_dummies(df, columns=['ProductCD'], prefix='ProductCD')

In [473]:
# card1
#df.card1.isna().sum()
# No changes at the beginning

In [474]:
# card2
#df.card2.isna().sum()
#df.card2.min()

df["card2_nan"] = df.card2.isna().astype(int)
df.card2.fillna(0.0, inplace=True)

In [475]:
# card3
#df.card3.isna().sum()
#df.card3.min()

df["card3_nan"] = df.card3.isna().astype(int)
df.card3.fillna(0.0, inplace=True)

In [476]:
# card4
#df.card4.isna().sum()

df = pd.get_dummies(df, columns=['card4'], prefix='card4', dummy_na=True)

In [477]:
# card5
#df.card5.isna().sum()
#df.card5.min()

df["card5_nan"] = df.card5.isna().astype(int)
df.card5.fillna(0.0, inplace=True)

In [478]:
# card6
#df.card6.isna().sum()

df = pd.get_dummies(df, columns=['card6'], prefix='card6', dummy_na=True)

In [479]:
# addr1
#df.addr1.isna().sum()

df["addr1_nan"] = df.addr1.isna().astype(int)
df.addr1.fillna(0.0, inplace=True)

In [480]:
# addr2
#df.addr2.isna().sum()
#df.addr2.value_counts()
#df.addr2.min()

df["addr2_nan"] = df.addr2.isna().astype(int)
df.addr2.fillna(0.0, inplace=True)

In [481]:
# dist1
#df.dist1

df["dist1_nan"] = df.dist1.isna().astype(int)
df.dist1.fillna(-1.0, inplace=True)

In [482]:
# dist2
#df.dist2.min()

df["dist2_nan"] = df.dist2.isna().astype(int)
df.dist2.fillna(-1.0, inplace=True)

In [483]:
# P_emaildomain
#df.P_emaildomain

df = pd.get_dummies(df, columns=['P_emaildomain'], prefix='P_emaildomain', dummy_na=True)

In [484]:
# R_emaildomain
#df.R_emaildomain

df = pd.get_dummies(df, columns=['R_emaildomain'], prefix='R_emaildomain', dummy_na=True)

In [485]:
# C1
#df.C1.isna().sum()
# No changes

In [486]:
# C2
#df.C2.isna().sum()
# No changes

In [487]:
# C3
#df.C3.isna().sum()
#df.C3.value_counts()
# No changes

In [488]:
# C4
#df.C4.isna().sum()
#df.C4.value_counts()
# No changes

In [489]:
# C5
#df.C5.isna().sum()
#df.C5.value_counts()
# No changes

In [490]:
# C6
#df.C6
#df.C6.isna().sum()
#df.C6.value_counts()
# No changes

In [491]:
# C7
#df.C7
#df.C7.isna().sum()
#df.C7.value_counts()
# No changes

In [492]:
# C8
#df.C8
#df.C8.isna().sum()
#df.C8.value_counts()
# No changes

In [493]:
# C9
#df.C9
#df.C9.isna().sum()
#df.C9.value_counts()
# No changes

In [494]:
# C10
#df.C10
#df.C10.isna().sum()
#df.C10.value_counts()
# No changes

In [495]:
# C11
#df.C11
#df.C11.isna().sum()
#df.C11.value_counts()
# No changes

In [496]:
# C12
#df.C12
#df.C12.isna().sum()
#df.C12.value_counts()
# No changes

In [497]:
# C13
#df.C13
#df.C13.isna().sum()
#df.C13.value_counts()
# No changes

In [498]:
# C14
#df.C14
#df.C14.isna().sum()
#df.C14.value_counts()
# No changes

In [499]:
# D1
#df.D1
#df.D1.isna().sum()

df["D1_nan"] = df.D1.isna().astype(int)
df.D1.fillna(-1.0, inplace=True)

In [500]:
# D2
#df.D2
#df.D2.min()

df["D2_nan"] = df.D2.isna().astype(int)
df.D2.fillna(-1.0, inplace=True)

In [501]:
# D3
#df.D3

df["D3_nan"] = df.D3.isna().astype(int)
df.D3.fillna(-1.0, inplace=True)

In [502]:
# D4
#df.D4

df["D4_nan"] = df.D4.isna().astype(int)
df.D4.fillna(-1.0, inplace=True)

In [503]:
# D5
#df.D5

df["D5_nan"] = df.D5.isna().astype(int)
df.D5.fillna(-1.0, inplace=True)

In [504]:
# D6
#df.D6
#df.D6.value_counts()

df["D6_nan"] = df.D6.isna().astype(int)
df.D6.fillna(-1.0, inplace=True)

In [505]:
# D7
#df.D7
#df.D7.value_counts()

df["D7_nan"] = df.D7.isna().astype(int)
df.D7.fillna(-1.0, inplace=True)

In [506]:
# D8
#df.D8
#df.D8.value_counts()

df["D8_nan"] = df.D8.isna().astype(int)
df.D8.fillna(-1.0, inplace=True)

In [507]:
# D9
#df.D9
#df.D9.value_counts()

df["D9_nan"] = df.D9.isna().astype(int)
df.D9.fillna(-1.0, inplace=True)

In [508]:
# D10
#df.D10

df["D10_nan"] = df.D10.isna().astype(int)
df.D10.fillna(-1.0, inplace=True)

In [509]:
# D11
#df.D11

df["D11_nan"] = df.D11.isna().astype(int)
df.D11.fillna(-1.0, inplace=True)

In [510]:
# D12
#df.D12
#df.D12.value_counts()

df["D12_nan"] = df.D12.isna().astype(int)
df.D12.fillna(-1.0, inplace=True)

In [511]:
# D13
#df.D13
#df.D13.value_counts()

df["D13_nan"] = df.D13.isna().astype(int)
df.D13.fillna(-1.0, inplace=True)

In [512]:
# D14
#df.D14
#df.D14.value_counts()

df["D14_nan"] = df.D14.isna().astype(int)
df.D14.fillna(-1.0, inplace=True)

In [513]:
# D15
#df.D15

df["D15_nan"] = df.D15.isna().astype(int)
df.D15.fillna(-1.0, inplace=True)

In [514]:
# M1
#df.M1
#df.M1.value_counts()

df = pd.get_dummies(df, columns=['M1'], prefix='M1', dummy_na=True)

In [515]:
# M2
#df.M2
#df.M2.value_counts()

df = pd.get_dummies(df, columns=['M2'], prefix='M2', dummy_na=True)

In [516]:
# M3
#df.M3

df = pd.get_dummies(df, columns=['M3'], prefix='M3', dummy_na=True)

In [517]:
# M4
#df.M4
#df.M4.value_counts()

df = pd.get_dummies(df, columns=['M4'], prefix='M4', dummy_na=True)

In [518]:
# M5
#df.M5

df = pd.get_dummies(df, columns=['M5'], prefix='M5', dummy_na=True)

In [519]:
# M6
#df.M6

df = pd.get_dummies(df, columns=['M6'], prefix='M6', dummy_na=True)

In [520]:
# M7
#df.M7

df = pd.get_dummies(df, columns=['M7'], prefix='M7', dummy_na=True)

In [521]:
# M8
#df.M8

df = pd.get_dummies(df, columns=['M8'], prefix='M8', dummy_na=True)

In [522]:
# M9
#df.M9

df = pd.get_dummies(df, columns=['M9'], prefix='M9', dummy_na=True)

In [523]:
#
# Process all V-columns at once
#

MAX_AMOUNT_OF_UNIQUE = 16

for i in range(1, 340):
    col = "V" + str(i)
    
    if (len(df[col].value_counts()) > MAX_AMOUNT_OF_UNIQUE):
        df[col + "_nan"] = df[col].isna().astype(int)
        df[col].fillna(-1.0, inplace=True)
    else:
        df = pd.get_dummies(df, columns=[col], prefix=col, dummy_na=True)

In [524]:
# What is the final shape?
df.shape

(590540, 1789)

### Scale Transactions

In [525]:
%%time

#
# TEMPORARY DELETE THE TransactionID. It will be needed to join the Identities info
#
del df['TransactionID']

columns = df.columns

scaler = StandardScaler(copy=False)
#scaler = MinMaxScaler(copy=False)

df = scaler.fit_transform(df)

C:\Users\peterp\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Wall time: 29 s


In [526]:
df = pd.DataFrame(df, columns=columns)
df.head(3)

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,...,V330_nan,V331_nan,V332_nan,V333_nan,V334_nan,V335_nan,V336_nan,V337_nan,V338_nan,V339_nan
0,0.002137,0.743044,0.000000,0.649351,0.599156,0.583333,0.852941,0.001944,0.0,0.000213,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.000900,0.100885,0.673333,0.649351,0.430380,0.601852,0.852941,0.000000,0.0,0.000213,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.001840,0.210566,0.816667,0.649351,0.700422,0.611111,0.852941,0.027997,0.0,0.000213,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Load Identities

In [ ]:
ids = pd.read_csv("train_identity.csv")
ids.shape

In [ ]:
ids.head()

### Data Wrangling for Identities

In [ ]:
# id_01
#ids.id_01.isna().sum()
#ids.id_01.value_counts()
# No changes

In [ ]:
# id_02
#ids.id_02.isna().sum()
#ids.id_02.max()

ids["id_02" + "_nan"] = ids["id_02"].isna().astype(int)
ids["id_02"].fillna(0.0, inplace=True)

In [ ]:
# id_03
#ids.id_03.isna().sum()
#ids.id_03.value_counts()

ids = pd.get_dummies(ids, columns=['id_03'], prefix='id_03', dummy_na=True)

In [ ]:
# id_04
#ids.id_04.isna().sum()
#ids.id_04.value_counts()

ids = pd.get_dummies(ids, columns=['id_04'], prefix='id_04', dummy_na=True)

In [ ]:
# id_05
#ids.id_05.isna().sum()
#ids.id_05.value_counts()

ids["id_05" + "_nan"] = ids["id_05"].isna().astype(int)
ids["id_05"].fillna(100.0, inplace=True)

In [ ]:
# id_06
#ids.id_06.isna().sum()
#ids.id_06.value_counts()

ids["id_06" + "_nan"] = ids["id_06"].isna().astype(int)
ids["id_06"].fillna(100.0, inplace=True)

In [ ]:
# id_07
#ids.id_07.isna().sum()
#ids.id_07.value_counts()
#ids.id_07.min()

ids["id_07" + "_nan"] = ids["id_07"].isna().astype(int)
ids["id_07"].fillna(100.0, inplace=True)

In [ ]:
# id_08
#ids.id_08.isna().sum()
#ids.id_08.value_counts()
#ids.id_08.max()

ids["id_08" + "_nan"] = ids["id_08"].isna().astype(int)
ids["id_08"].fillna(100.0, inplace=True)

In [ ]:
# id_09
#ids.id_09.isna().sum()
#ids.id_09.value_counts()

ids["id_09" + "_nan"] = ids["id_09"].isna().astype(int)
ids["id_09"].fillna(100.0, inplace=True)

In [ ]:
# id_10
#ids.id_10.isna().sum()
#ids.id_10.value_counts()

ids["id_10" + "_nan"] = ids["id_10"].isna().astype(int)
ids["id_10"].fillna(100.0, inplace=True)

In [ ]:
# id_11
#ids.id_11.isna().sum()
#ids.id_11.value_counts()
#ids.id_11.min()

ids["id_11" + "_nan"] = ids["id_11"].isna().astype(int)
ids["id_11"].fillna(0.0, inplace=True)

In [ ]:
# id_12
#ids.id_12.isna().sum()
#ids.id_12.value_counts()

ids = pd.get_dummies(ids, columns=['id_12'], prefix='id_12', dummy_na=True)

In [ ]:
# id_13
#ids.id_13.isna().sum()
#ids.id_13.value_counts()

ids["id_13" + "_nan"] = ids["id_13"].isna().astype(int)
ids["id_13"].fillna(100.0, inplace=True)

In [ ]:
# id_14
#ids.id_14.isna().sum()
#ids.id_14.value_counts()

ids = pd.get_dummies(ids, columns=['id_14'], prefix='id_14', dummy_na=True)

In [ ]:
# id_15
#ids.id_15.isna().sum()
#ids.id_15.value_counts()

ids = pd.get_dummies(ids, columns=['id_15'], prefix='id_15', dummy_na=True)

In [ ]:
# id_16
#ids.id_16.isna().sum()
#ids.id_16.value_counts()

ids = pd.get_dummies(ids, columns=['id_16'], prefix='id_16', dummy_na=True)

In [ ]:
# id_17
#ids.id_17.isna().sum()
#ids.id_17.value_counts()
#ids.id_17.min()

ids["id_17" + "_nan"] = ids["id_17"].isna().astype(int)
ids["id_17"].fillna(0.0, inplace=True)

In [ ]:
# id_18
#ids.id_18.isna().sum()
#ids.id_18.value_counts()

ids = pd.get_dummies(ids, columns=['id_18'], prefix='id_18', dummy_na=True)

In [ ]:
# id_19
#ids.id_19.isna().sum()
#ids.id_19.value_counts()
#ids.id_19.min()

ids["id_19" + "_nan"] = ids["id_19"].isna().astype(int)
ids["id_19"].fillna(0.0, inplace=True)

In [ ]:
# id_20
#ids.id_20.isna().sum()
#ids.id_20.value_counts()
#ids.id_20.min()

ids["id_20" + "_nan"] = ids["id_20"].isna().astype(int)
ids["id_20"].fillna(0.0, inplace=True)

In [ ]:
# id_21
#ids.id_21.isna().sum()
#ids.id_21.value_counts()
#ids.id_21.min()

ids["id_21" + "_nan"] = ids["id_21"].isna().astype(int)
ids["id_21"].fillna(0.0, inplace=True)

In [ ]:
# id_22
#ids.id_22.isna().sum()
#ids.id_22.value_counts()

ids = pd.get_dummies(ids, columns=['id_22'], prefix='id_22', dummy_na=True)

In [ ]:
# id_23
#ids.id_23.isna().sum()
#ids.id_23.value_counts()

ids = pd.get_dummies(ids, columns=['id_23'], prefix='id_23', dummy_na=True)

In [ ]:
# id_24
#ids.id_24.isna().sum()
#ids.id_24.value_counts()

ids = pd.get_dummies(ids, columns=['id_24'], prefix='id_24', dummy_na=True)

In [ ]:
# id_25
#ids.id_25.isna().sum()
#ids.id_25.value_counts()
#ids.id_25.min()

ids["id_25" + "_nan"] = ids["id_25"].isna().astype(int)
ids["id_25"].fillna(0.0, inplace=True)

In [ ]:
# id_26
#ids.id_26.isna().sum()
#ids.id_26.value_counts()
#ids.id_26.min()

ids["id_26" + "_nan"] = ids["id_26"].isna().astype(int)
ids["id_26"].fillna(0.0, inplace=True)

In [ ]:
# id_27
#ids.id_27.isna().sum()
#ids.id_27.value_counts()

ids = pd.get_dummies(ids, columns=['id_27'], prefix='id_27', dummy_na=True)

In [ ]:
# id_28
#ids.id_28.isna().sum()
#ids.id_28.value_counts()

ids = pd.get_dummies(ids, columns=['id_28'], prefix='id_28', dummy_na=True)

In [ ]:
# id_29
#ids.id_29.isna().sum()
#ids.id_29.value_counts()

ids = pd.get_dummies(ids, columns=['id_29'], prefix='id_29', dummy_na=True)

In [ ]:
# id_30
#ids.id_30.isna().sum()
#ids.id_30.value_counts()

ids["id_30"].fillna("Missing", inplace=True)
ids["OS_Type"] = ids.id_30.str.split(" ").map(lambda x: x[0])

ids = pd.get_dummies(ids, columns=['id_30'], prefix='id_30', dummy_na=True)
ids = pd.get_dummies(ids, columns=['OS_Type'], prefix='OS_Type', dummy_na=True)

In [ ]:
# id_31
#ids.id_31.isna().sum()
#list(ids.id_31.value_counts().index)

ids["id_31"].fillna("Missing", inplace=True)

browser_type = []

for i in range(ids.shape[0]):
    browser = ids.loc[i, "id_31"].lower()
    
    if ("chrome" in browser):
        browser_type.append("chrome")
    elif("safari" in browser):
        browser_type.append("safari")
    elif("ie" in browser):
        browser_type.append("ie")    
    elif("safari" in browser):
        browser_type.append("safari")
    elif("firefox" in browser):
        browser_type.append("firefox")      
    elif("samsung" in browser):
        browser_type.append("samsung")   
    elif("edge" in browser):
        browser_type.append("edge")   
    elif("webview" in browser):
        browser_type.append("webview") 
    elif("opera" in browser):
        browser_type.append("opera")         
    elif("search" in browser):
        browser_type.append("search")     
    elif("microsoft" in browser):
        browser_type.append("microsoft") 
    else:
        browser_type.append(browser)
        
ids = pd.get_dummies(ids, columns=['id_31'], prefix='id_31', dummy_na=True)
ids['Browser_Type'] = browser_type

In [ ]:
# id_32
#ids.id_32.isna().sum()
#ids.id_32.value_counts()

ids = pd.get_dummies(ids, columns=['id_32'], prefix='id_32', dummy_na=True)

In [ ]:
# id_33
#ids.id_33.isna().sum()
#ids.id_33.value_counts()

ids = pd.get_dummies(ids, columns=['id_33'], prefix='id_33', dummy_na=True)

In [ ]:
# id_34
#ids.id_34.isna().sum()
#ids.id_34.value_counts()

ids = pd.get_dummies(ids, columns=['id_34'], prefix='id_34', dummy_na=True)

In [ ]:
# id_35
#ids.id_35.isna().sum()
#ids.id_35.value_counts()

ids = pd.get_dummies(ids, columns=['id_35'], prefix='id_35', dummy_na=True)

In [ ]:
# id_36
#ids.id_36.isna().sum()
#ids.id_36.value_counts()

ids = pd.get_dummies(ids, columns=['id_36'], prefix='id_36', dummy_na=True)

In [ ]:
# id_37
#ids.id_37.isna().sum()
#ids.id_37.value_counts()

ids = pd.get_dummies(ids, columns=['id_37'], prefix='id_37', dummy_na=True)

In [ ]:
# id_38
#ids.id_38.isna().sum()
#ids.id_38.value_counts()

ids = pd.get_dummies(ids, columns=['id_38'], prefix='id_38', dummy_na=True)

### Merge Identities and Transactions

In [ ]:
data = pd.merge(df, ids, on="TransactionID", how="left")
data.shape

In [ ]:
data.head()

In [ ]:
del data["TransactionID"]

In [ ]:
ids.id_01.isna().sum()

In [ ]:
data.id_01.isna().sum()

#### not clear yet what to do after merge.
#### only 145K rows will be full
#### let's first look at the performance on the Transactions set without Identities

### Split to Train and Test Sets

In [527]:
X_train, X_test, y_train, y_test = train_test_split(df, labels, stratify=labels, test_size=0.3, random_state=8)

### Gradient Boosting Classifier

In [ ]:
%%time
#
# Gradient Boosting Machine
#
GBR = GradientBoostingRegressor(random_state=8)

parameters_grid = {
    "n_estimators": [350, 400, 450],
    "min_samples_split": [2, 3, 4],
    "min_samples_leaf": [1, 2, 3, 4]
}

gcv = GridSearchCV(GBR, parameters_grid, scoring='neg_mean_squared_error')
gcv.fit(features_train, labels_train)
GBR = gcv.best_estimator_
print(GBR)
print("GBR Score:", gcv.best_score_)

### Neural Network

In [ ]:
df.shape

In [532]:
def rocauc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)


classifier = Sequential()

classifier.add(Dense(512, activation='relu', kernel_initializer='normal', input_dim=df.shape[1]))
classifier.add(Dropout(0.1))
classifier.add(Dense(128, activation='relu', kernel_initializer='normal'))
classifier.add(Dropout(0.1))
classifier.add(Dense(16, activation='relu', kernel_initializer='normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='normal'))

classifier.compile(optimizer ='adam', loss='binary_crossentropy', metrics =['accuracy', rocauc])

classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 512)               915968    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 128)               65664     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 16)                2064      
_________________________________________________________________
dense_106 (Dense)            (None, 1)                 17        
Total params: 983,713
Trainable params: 983,713
Non-trainable params: 0
_________________________________________________________________


In [529]:
X_train_1 = X_train[y_train.astype(bool).values]
X_train_1['Label'] = 1

C:\Users\peterp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [530]:
X_train_0 = X_train[(1 - y_train).astype(bool).values]
X_train_0['Label'] = 0

C:\Users\peterp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
X_train_0.shape

In [533]:
%%time

for _ in range(50):
    sample_X_zeros = X_train_0.sample(14464)
    temp_X_train = pd.concat([sample_X_zeros, X_train_1])
    temp_X_train = temp_X_train.sample(frac=1).reset_index(drop=True)
    
    temp_y_train = temp_X_train['Label']
    del temp_X_train['Label']
    
    classifier.fit(temp_X_train, temp_y_train, batch_size=512, epochs=5)

# NN Performance check
print("Accuracy on Test set:")
predictions = classifier.predict(X_test)
print(roc_auc_score(y_test, np.around(predictions)))

Epoch 1/5
28928/28928 [==============================] - 3s 117us/sample - loss: 0.5791 - acc: 0.6995 - rocauc: 0.7747 - loss: 0.5918 - acc: 0.6887 - rocauc: 0.76 - ETA: 0s - loss: 0.5911 - acc: 0.6895 - 
Epoch 2/5
28928/28928 [==============================] - 3s 91us/sample - loss: 0.4994 - acc: 0.7571 - rocauc: 0.8371
Epoch 3/5
28928/28928 [==============================] - 3s 93us/sample - loss: 0.4799 - acc: 0.7673 - rocauc: 0.8504
Epoch 4/5
28928/28928 [==============================] - 3s 97us/sample - loss: 0.4676 - acc: 0.7763 - rocauc: 0.8574
Epoch 5/5
28928/28928 [==============================] - 3s 92us/sample - loss: 0.4658 - acc: 0.7744 - rocauc: 0.8607
Epoch 1/5
28928/28928 [==============================] - 3s 89us/sample - loss: 0.4589 - acc: 0.7844 - rocauc: 0.8619
Epoch 2/5
28928/28928 [==============================] - 3s 93us/sample - loss: 0.4550 - acc: 0.7825 - rocauc: 0.8670
Epoch 3/5
28928/28928 [==============================] - 3s 93us/sample - loss: 0.4508 

28928/28928 [==============================] - 3s 93us/sample - loss: 0.3271 - acc: 0.8523 - rocauc: 0.9333
Epoch 4/5
28928/28928 [==============================] - 3s 99us/sample - loss: 0.3218 - acc: 0.8556 - rocauc: 0.9354
Epoch 5/5
28928/28928 [==============================] - 3s 94us/sample - loss: 0.3240 - acc: 0.8529 - rocauc: 0.9345
Epoch 1/5
28928/28928 [==============================] - 2s 86us/sample - loss: 0.3378 - acc: 0.8467 - rocauc: 0.9300
Epoch 2/5
28928/28928 [==============================] - 3s 89us/sample - loss: 0.3264 - acc: 0.8507 - rocauc: 0.9339
Epoch 3/5
28928/28928 [==============================] - 3s 98us/sample - loss: 0.3180 - acc: 0.8558 - rocauc: 0.9375
Epoch 4/5
28928/28928 [==============================] - 3s 94us/sample - loss: 0.3135 - acc: 0.8566 - rocauc: 0.9391
Epoch 5/5
28928/28928 [==============================] - 3s 94us/sample - loss: 0.3155 - acc: 0.8575 - rocauc: 0.9379
Epoch 1/5
28928/28928 [==============================] - 3s 88us/s

Epoch 1/5
28928/28928 [==============================] - 3s 91us/sample - loss: 0.2900 - acc: 0.8712 - rocauc: 0.9484
Epoch 2/5
28928/28928 [==============================] - 3s 95us/sample - loss: 0.2808 - acc: 0.8735 - rocauc: 0.9515
Epoch 3/5
28928/28928 [==============================] - 3s 92us/sample - loss: 0.2803 - acc: 0.8720 - rocauc: 0.9521
Epoch 4/5
28928/28928 [==============================] - 3s 91us/sample - loss: 0.2724 - acc: 0.8764 - rocauc: 0.9543s - loss: 0.2722 - acc: 0.8
Epoch 5/5
28928/28928 [==============================] - 3s 101us/sample - loss: 0.2687 - acc: 0.8765 - rocauc: 0.9554
Epoch 1/5
28928/28928 [==============================] - 3s 87us/sample - loss: 0.2821 - acc: 0.8726 - rocauc: 0.9512
Epoch 2/5
28928/28928 [==============================] - 3s 90us/sample - loss: 0.2768 - acc: 0.8754 - rocauc: 0.9527
Epoch 3/5
28928/28928 [==============================] - 3s 93us/sample - loss: 0.2709 - acc: 0.8795 - rocauc: 0.9548
Epoch 4/5
28928/28928 [=====

28928/28928 [==============================] - 3s 95us/sample - loss: 0.2469 - acc: 0.8900 - rocauc: 0.9628
Epoch 4/5
28928/28928 [==============================] - 3s 92us/sample - loss: 0.2407 - acc: 0.8933 - rocauc: 0.9645
Epoch 5/5
28928/28928 [==============================] - 3s 92us/sample - loss: 0.2409 - acc: 0.8916 - rocauc: 0.9642
Epoch 1/5
28928/28928 [==============================] - 3s 89us/sample - loss: 0.2589 - acc: 0.8847 - rocauc: 0.9591
Epoch 2/5
28928/28928 [==============================] - 3s 92us/sample - loss: 0.2496 - acc: 0.8880 - rocauc: 0.9620s - l
Epoch 3/5
28928/28928 [==============================] - 3s 91us/sample - loss: 0.2454 - acc: 0.8897 - rocauc: 0.9634
Epoch 4/5
28928/28928 [==============================] - 3s 91us/sample - loss: 0.2434 - acc: 0.8900 - rocauc: 0.9642
Epoch 5/5
28928/28928 [==============================] - 3s 99us/sample - loss: 0.2420 - acc: 0.8922 - rocauc: 0.9641
Epoch 1/5
28928/28928 [==============================] - 2s 8

In [ ]:
predictions = classifier.predict(X_test)

print(accuracy_score(y_test, np.around(predictions)))

In [ ]:


print(accuracy_score(y_test, np.around(predictions)))

In [ ]:
count = 0

for x in predictions:
    if (x > 0.5):
        count = count + 1
        
print(count)

In [ ]:
temp_X_train